[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/company_hierarchy.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/company_hierarchy.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fcompany_hierarchy.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] |
|:---------------------:|:---------------:|:-------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It allows the user to visualise the company hierarchy on the system, in three different ways:
* As a [Pandas](https://pandas.pydata.org/) table;
* As a [NetworkX](https://networkx.github.io/) graph object;
* As an interactive [bqplot](https://bqplot.readthedocs.io/en/latest/) graph.

### Requirements
* If you are running in Jupyter Notebook, you don't need to make any code changes.
* If you are running in Jupyter Lab, you will also need to install these JupyterLab extensions:
  * @jupyter-widgets/jupyterlab-manager
  * bqplot
* You need to have an existing **SysAdmin** account on the WA/DMP instance.

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells one at a time (Shift-Enter), or all at once.

Some cells prompt for user input.

## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install ipywidgets

# We have had issues with bqplot due to version incompatibilities.
# To be safe, we pin both the backend (pip) version and the frontend (npm) version.
!{sys.executable} -m pip install bqplot==0.12.6
# bqplot includes ipywidgets, numpy, pandas, so we don't need to install those.

# The following line is commented out because it will cause an exception in Jupyter Notebook.
# HOWEVER, it may be required in Jupyter Lab if you can't install the right version through the Extension Manager.
#!jupyter labextension install bqplot@0.5.6

import requests
import json
import pandas as pd
import networkx as nx
import ipywidgets
import bqplot

## Global variables

In [ ]:
BASE_URL = 'https://gateway.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

## Functions to be used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = SESSION.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_companies(name=None):
    """Gets the list of companies in the system.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies"
    query = {'name': name}
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of companies!")
        return None

    
def get_users(email=None, companies=[]):
    """Gets the list of users in a company or companies.
    """
    url = f"{BASE_URL}/{BASE_PATH}/users"
    query = {'email': email, 'companies': companies}
    print(f"Sending GET request to {url} with:\n"
          f"    email={email}\n"
          f"    companies={companies}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of users!")
        return None

## Login to server

In [ ]:
USERNAME = input("Enter WebAccess/DMP username:")
PASSWORD = input("Enter password:")
user_token = login(USERNAME, PASSWORD)
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Get Companies

In [ ]:
companies = get_companies()

## Create a table

In [ ]:
# A nested list comprehension creates a 2D array.
# We can't include "Parent Name" information yet, because a company may not have a parent
data = [[
    company['name'],
    company['id'],
    company['contact_name']
    ] for company in companies]

# Convert the 2D array into a Pandas dataframe, because it renders nicely
table = pd.DataFrame(data, columns=["name", "id", "contact_name"])

parent_column = []
users_column = []

for company in companies:
    num_users = len(get_users(companies=[company['id']]))
    users_column.append(num_users)
    if company['parent']:
        parent_column.append(company['parent']['name'])
    else:
        parent_column.append('')

In [ ]:
# Add new columns
table['Parent Name'] = parent_column
table['Number of Users'] = users_column
table

## Save to CSV file
If you are running this notebook on your local machine, the file will automatically be saved to the same directory where this notebook is located.

If you are running in a hosted environment like Binder or Google Colab, the file will be saved to the filesystem of the virtual machine!
  * In the case of Binder, there is no (easy) way to download the file.
  * In the case of Google Colab, there is a "Files" icon in the left-hand sidebar. Click here to find your file and dowload it.

In [ ]:
table.to_csv("output.csv")

## Create a NetworkX graph
References:
* https://networkx.github.io/documentation/networkx-2.3/index.html
* https://networkx.github.io/documentation/networkx-2.3/reference/algorithms/tree.html

In [ ]:
DG = nx.DiGraph(server=BASE_URL)

# Add each company as a node
for i, company in enumerate(companies):
    DG.add_node(
        company['name'],
        # Add extra attributes
        id = company['id'],
        contact_name = company['contact_name'],
        users = table.at[i, 'Number of Users']
    )
    
# Add parent-child relationships as edges
for company in companies:
    if company['parent']:
        DG.add_edge(company['parent']['name'], company['name'])

print(f"Tree: {nx.algorithms.tree.recognition.is_tree(DG)}")
print(f"Forest: {nx.algorithms.tree.recognition.is_forest(DG)}")
print(f"Arborescence: {nx.algorithms.tree.recognition.is_arborescence(DG)}")
print(f"Branching: {nx.algorithms.tree.recognition.is_branching(DG)}")

## Create a graph in bqplot

In [ ]:
fig_layout = ipywidgets.Layout(width='960px', height='500px')

nodes = list(DG.nodes)
edges = [{'source': s, 'target': t} for s, t in list(DG.edges)]
#print(nodes)
#print(edges)

# Add some extra attributes to the nodes. We can use these in tooltips.
nodes2 = []
for i, node in enumerate(nodes):
    num_users = table.at[i, 'Number of Users']
    # We want the circle's AREA to be proportional to the Number of Users
    radius = (num_users*50)**0.5
    nodes2.append(
        {'label': node,
         'name': node,
         'id': table.at[i, 'id'],
         'users': num_users,
         'children': len(list(DG.successors(node))),
         'shape': 'circle',
         'shape_attrs': {'r': radius}
        }
    )
#print(nodes2)

# Unfortunately, the bqplot graph can only use integers to identify the nodes in link_data, not strings.
# So we look up the company names in the table and take the corresponding row index:
links = []
for edge in edge_data:
    source_id = table.query(f"name == '{edge['source']}'").index.tolist()[0]
    target_id = table.query(f"name == '{edge['target']}'").index.tolist()[0]
    links.append({'source': source_id, 'target': target_id})
#print(links)

In [ ]:
graph = bqplot.Graph(node_data=nodes2, link_data=links, link_type='line', link_distance=100, charge=-100)

tooltip = bqplot.Tooltip(fields=['name', 'id', 'users', 'children'], formats=['', '0d', '0d', '0d'])
graph.tooltip = tooltip

figure = bqplot.Figure(marks=[graph], layout=fig_layout)
figure

Depending on the size of the graph, you may need to adjust the `link_distance` and `charge` attributes in the cell immediately above.

If the company circles are not a good size, adjust the `radius` calculation in the cell two above.

## Save to PNG file
*As noted above for output.csv:*

If you are running this notebook on your local machine, the file will automatically be saved to the same directory where this notebook is located.

If you are running in a hosted environment like Binder or Google Colab, the file will be saved to the filesystem of the virtual machine!
* In the case of Binder, there is no (easy) way to download the file.
* In the case of Google Colab, there is a "Files" icon in the left-hand sidebar. Click here to find your file and dowload it.

In [ ]:
figure.save_png('output.png')